<a href="https://colab.research.google.com/github/ccwu0918/deeplearning.ai/blob/main/w7-l5_Passive_and_active_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 5: Passive and active monitoring

## Setup

In [28]:
!git clone https://github.com/ccwu0918/deeplearning.ai

Cloning into 'deeplearning.ai'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 104 (delta 26), reused 0 (delta 0), pack-reused 55
Receiving objects: 100% (104/104), 492.87 KiB | 7.95 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [29]:
!pip install -q whylogs

In [30]:
!pip install -q langkit[all]

In [ ]:
!pip install -q whylogs[viz]

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
# !pip install -q span_marker

In [ ]:
# !pip install -q bert_score

In [4]:
%cd /content/deeplearning.ai/extras

/content/deeplearning.ai/extras


In [5]:
import pandas as pd

In [6]:
import whylogs as why

In [7]:
import helpers

In [8]:
from langkit import llm_metrics

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [9]:
llm_metrics.init()

In [10]:
from whylogs.experimental.core.udf_schema import register_dataset_udf

In [11]:
from whylogs.experimental.core.udf_schema import udf_schema

In [12]:
llm_schema = udf_schema()

In [13]:
llm_logger = why.logger(schema=udf_schema())

In [14]:
llm_logger = why.logger(
    model = "rolling",
    interval = 1,
    when = "H",
    schema = udf_schema()
)

**Note**: To accessthe WhyLabs platform that was built in the previous lessons: https://hub.whylabsapp.com/resources/model-1/profiles?profile=ref-EBT5yDFL0lyq0r93&sessionToken=session-pPdc5R9m

## Building your own active monitoring guardrails

In [17]:
!pip install -q openai

In [18]:
!pip install --quiet python-dotenv

In [19]:
import openai

In [21]:
openai.api_key = helpers.get_openai_key()
openai.base_url = helpers.get_openai_base_url()

In [22]:
import os
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai.api_key

In [25]:
# print(os.getenv("OPENAI_API_KEY"))

In [26]:
active_llm_logger = why.logger()

In [27]:
def user_request():
    # Take request
    request = input("\nEnter your desired item to make a recipe" \
                    "(or 'quit'):")
    if request.lower() == "quit":
        raise KeyboardInterrupt()

    # Log request
    active_llm_logger.log({"request": request})

    return request

In [34]:
def prompt_llm(request):
    # Transform prompt
    prompt = f"""Please give me a short recipe for creating"\
    the following item in up to 6 steps. Each step of the recipe "\
    should be summarized in no more than 200 characters."\
    Item: {request}"""

    # Log prompt
    active_llm_logger.log({"prompt": prompt})

    # Collect response from LLM
    # response = openai.ChatCompletion.create(
    response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [{
            "role": "system",
            "content": prompt
        }]
    # )["choices"][0]["message"]["content"]
    ).choices[0].message.content

    # Log response
    active_llm_logger.log({"response": response})

    return response

In [35]:
def user_reply_success(request,response):
    # Create and print user reply
    reply = f"\nSuccess! Here is the recipe for"\
            f"{request}:\n{response}"
    print(reply)

    #Log reply
    active_llm_logger.log({"reply": reply})

In [36]:
def user_reply_failure(request = "your request"):
    # Create and print user reply
    reply = ("\nUnfortunately, we are not able to provide a recipe for " \
            f"{request} at this time. Please try Recipe Creator 900 " \
            f"in the future.")
    print(reply)

    #Log reply
    active_llm_logger.log({"reply": reply})

In [37]:
class LLMApplicationValidationError(ValueError):
    pass

In [49]:
while True:
    try:
        request = user_request()
        response = prompt_llm(request)
        user_reply_success(request, response)
    except KeyboardInterrupt:
        break
    except LLMApplicationValidationError:
        user_reply_failure(request)
        break


Enter your desired item to make a recipe(or 'quit'):Steak dishes

Success! Here is the recipe forSteak dishes:
Step 1: Preheat grill to high heat. Season steaks with salt, pepper, and desired spices. 
Step 2: Grill steaks for 4-5 minutes per side for medium-rare, adjusting cooking time based on desired doneness. 
Step 3: Remove steaks from grill and let rest for 5 minutes to allow juices to redistribute. 
Step 4: In a saucepan, melt butter and sauté garlic until fragrant. 
Step 5: Slice steaks against the grain and serve with garlic butter sauce. 
Step 6: Garnish with fresh herbs and enjoy your flavorful steak dish.

Enter your desired item to make a recipe(or 'quit'):quit


In [39]:
from whylogs.core.relations import Predicate
from whylogs.core.metrics.condition_count_metric import Condition
from whylogs.core.validators import ConditionValidator

In [40]:
def raise_error(validator_name, condition_name, value):
    raise LLMApplicationValidationError(
        f"Failed {validator_name} with value {value}."
    )

In [41]:
low_condition = {"<0.3": Condition(Predicate().less_than(0.3))}

In [42]:
toxicity_validator = ConditionValidator(
    name = "Toxic",
    conditions = low_condition,
    actions = [raise_error]
)

In [43]:
refusal_validator = ConditionValidator(
    name = "Refusal",
    conditions = low_condition,
    actions = [raise_error]
)

In [44]:
llm_validators = {
    "prompt.toxicity": [toxicity_validator],
    "response.refusal_similarity": [refusal_validator]
}

In [45]:
active_llm_logger = why.logger(
    model = "rolling",
    interval = 5,
    when = "M",
    base_name = "active_llm",
    schema = udf_schema(validators = llm_validators)
)

In [48]:
active_llm_logger.log(
    {"response":"I'm sorry, but I can't answer that."}
)

LLMApplicationValidationError: Failed Refusal with value 0.5789186954498291.

 ⚠️ **Disclaimer**: Please be aware that the code may not capture all safety concerns and some undesired responses can still pass through. We encourage you to explore ways in which you can make the monitoring system more robust.

In [47]:
while True:
    try:
        request = user_request()
        response = prompt_llm(request)
        user_reply_success(request, response)
    except KeyboardInterrupt:
        break
    except LLMApplicationValidationError:
        user_reply_failure(request)
        break


Enter your desired item to make a recipe(or 'quit'):pasta

Success! Here is the recipe forpasta:
1. Boil water in a pot, add salt and cook pasta until al dente.
2. In a separate pan, heat olive oil and sauté garlic with desired herbs.
3. Add tomato sauce or favorite pasta sauce to the pan.
4. Drain cooked pasta, then toss it into the sauce pan, mixing well.
5. Let the pasta simmer for a few minutes to absorb flavors.
6. Serve hot, garnish with grated cheese and fresh herbs if desired. Enjoy!

Enter your desired item to make a recipe(or 'quit'):quit
